# Connecting speakers, ideology and phonetics

We form our working phonetics-based dataset. 

In [1]:
formants=open("gathered_data/wordssyllablesformants.txt",'r')
list_formants=formants.readlines()

- **Data cleaning.** First, we reformat the data produced by the gathering scripts. 
We form a dictionary whose keys are tuples (speaker, word, vowel, position_in_the_word) and values are the phonetics data, namely a list $[\text{frequence 1}, \text{frequence 2}, \text{duration}, \text{formant1}, \text{formant 2}]$ where both formant1 and formant2 are themselves lists containing 3 frequences.
The script also populates a list of all the speakers.

In [2]:
def reformat(l,vowels_dict,names_list):
    l=l.replace('[','')
    l=l.replace(']','')
    l=l.replace("'",'')
    l=l.replace(' ','')
    l=l.split(',')
    word=l[0]
    speaker=l[1].lower()
    for i in range(3,len(l),13):
        position = ((i-3)/13)+1
        sublist=l[i:i+12]
        vowel=sublist[0]
        freq1=float(sublist[1])
        freq2=float(sublist[2])
        duration=float(sublist[3])
        formant1=[float(i) for i in sublist[4:7]]
        formant2=[float(i) for i in sublist[8:11]]
        if (speaker,word,vowel,position) in vowels_dict:
            v=vowels_dict[(speaker,word,vowel,position)]
            v[0]+=1
            v[1]+=freq1
            v[2]+=freq2
            v[3]+=duration
            for i in range(len(formant1)):
                v[4][i]+=formant1[i]
                v[5][i]+=formant2[i]

        else:
            vowels_dict[(speaker,word,vowel,position)]=[1,freq1,freq2,duration,formant1,formant2]    
    names_list.append(speaker)
    
vowels_dict={}
names_list=[]
for line in list_formants:
    reformat(line,vowels_dict,names_list)

names_list=list(set(names_list)) #in order to remove duplicates
len(names_list)

804

- **Averaging the formants.** This is an important step. 
We average the phonetics data for each key of our dictionary. This in order the future data analysis to overestimate the role of certain "dominant" speakers in the ideology prediction. As a result, we still have a dictionary whose keys are (speaker, word, vowel), but the value is now a single, averaged phonetics information.

In [3]:
for e in vowels_dict:
    v=vowels_dict[e]
    count=v[0]
    v[1]=v[1]/count
    v[2]=v[2]/count
    v[3]=v[3]/count
    formant1=v[4]
    for i in range(len(formant1)):
        v[4][i]=v[4][i]/count
        v[5][i]=v[5][i]/count

- **Getting the ideology for each speaker.**
However, not all the speakers appear in ideodefined.txt as the donations record for some speakers either weren't found or don't allow to conclude regarding their political orientation.
We form a dictionary where keys are (first, last name) and values are the ideology. We find 433 different speakers with ideology.

In [4]:
ideology=open("gathered_data/ideodefined.txt",'r')  #file containing speakers and their respective ideology. 
list_ideology=ideology.readlines()
ideology_dict={}
names=[]
for e in list_ideology:
    l=e[:-1].split(',')
    first_name=l[0]
    last_name=l[1]
    ideo=l[2]
    ideology_dict[(first_name,last_name)]=ideo
    names.append((first_name,last_name))

We also create a dictionary to match names (as they do not have the same format in both files)

In [5]:
wordsyllablesformant_names=list(set(names_list[:]))
ideology_names=list(set(names[:]))
match={}
for m in ideology_names:
    for n in wordsyllablesformant_names:
        if (m[0] in n and m[1].replace("'",'') in n):  #replace the ' by a blank for Irish names
            wordsyllablesformant_names.remove(n)
            match[n]=m

- **Replacing names by ideology in the phonetics dataset**

In [6]:
vowels_ideology={}
keys=[]
for e in vowels_dict:
    keys.append(e)
for k in keys:
    speaker=k[0]
    word=k[1]
    vowel=k[2]
    position=k[3]
    v=vowels_dict[k]
    try:
        sep_name=match[speaker]
        ideology_of_speaker=ideology_dict[sep_name]
        vowels_ideology[(speaker,ideology_of_speaker,word,vowel,position)]=v
    except KeyError:
        oops=1

We do some more data formatting so that the values in the dictionary are as follows

In [7]:
col=['count','formant1','formant2','duration','F11','F12','F13','F21','F22','F23']

In [8]:
vowels_final={}
for e in vowels_ideology:
    l=[]
    v=vowels_ideology[e]
    l=v[0:4]
    for i in v[4]:
        l.append(i)
    for j in v[5]:
        l.append(j)
    vowels_final[e]=l

- ** Most used words ** We list the words by decreasing frequency, we select the top 20 words (that do not seem to be ideologically relevant) and we arbitrarily pick the first 20 words that may have some ideological content.

In [9]:
count={}
for e in vowels_final:
    try:
        count[e[1]]+=1
    except KeyError:
        count[e[1]]=1

import operator
sorted_count = sorted(count.items(), key=operator.itemgetter(1),)
sorted(count.items(), key=lambda x: -x[1])

[('0.0', 351019),
 ('1.0', 119021),
 ('0.26754289597142034', 27276),
 ('0.28797816531883524', 15416),
 ('0.9901960784313726', 9744),
 ('0.11904761904761904', 7987),
 ('0.9056603773584906', 4714),
 ('0.989451476793249', 4582),
 ('0.9', 4579),
 ('0.3638457203316736', 3866),
 ('0.4888888888888889', 3528),
 ('0.8685759145912924', 3505),
 ('0.4473684210526316', 3421),
 ('0.08950545727963642', 3200),
 ('0.5', 2949),
 ('0.047573739295908656', 2814),
 ('0.037286724927174365', 2724),
 ('0.9199999999999999', 2638),
 ('0.39601850516956444', 2593),
 ('0.4304347826086956', 2546),
 ('0.9768518518518519', 2354),
 ('0.9985696771557009', 2274),
 ('0.9324324324324325', 2205),
 ('0.23226773226773229', 2132),
 ('0.002472799208704253', 2081),
 ('0.03540179922937689', 2059),
 ('0.20978831347592225', 1982),
 ('0.4949186991869919', 1952),
 ('0.3858168631654727', 1945),
 ('0.7004124065641378', 1842),
 ('0.3333333333333333', 1811),
 ('0.43993552669005404', 1800),
 ('0.3975563909774436', 1691),
 ('0.571428571428

In [10]:
non_charged=['SITUATION','BECAUSE','EXAMPLE','PARTICULAR','DIFFERENT','CERTAINLY',
             'IMPORTANT','INTO','ABSOLUTELY','ABOUT','UNDER','SPECIFICALLY','WHETHER',
            'OTHER','EXACTLY']
charged=['FEDERAL','SCALIA','OPPORTUNITY','STATUTORY','HONOR','MR','LEGISLATIVE',
         'CONSTITUTIONAL','KENNEDY','AUTHORITY','GINSBURG','LEGISLATURE','PEOPLE',
        'POLICY','CONGRESS','GOVERNMENT','AMENDMENT','CONSTITUTION','ECONOMIC',
         'CALIFORNIA']

**Finally, we generate the dataset**: one file per vowel in a word (charged or uncharged), and each file contains a list of
ideology, phonetics_data
where phonetics_data is
'count','formant1','formant2','duration','F11','F12','F13','F21','F22','F23'

In [14]:
for entry in vowels_final:
    ideology,word,vowel,position = entry
    if word in non_charged:
        thisword = open("gathered_data/non_charged/"+str(word)+'_'+str(vowel)+'_'+str(position)+'.txt', 'a')
        thisword.write(str(ideology)+','+str(vowels_final[entry]).replace('[', '').replace(']','')+'\n')
        thisword.close()
    elif word in charged:
        thisword = open("gathered_data/charged/"+str(word)+'_'+str(vowel)+'_'+str(position)+'.txt', 'a')
        thisword.write(str(ideology)+','+str(vowels_final[entry]).replace('[', '').replace(']','')+'\n')
        thisword.close()
    else:
        pass

Count the number of different speakers per word

In [17]:
word_speakers = {}
word_count = {}
for word in charged:
    word_speakers[word] = {}
for word in non_charged:
    word_speakers[word] = {}
    
for entry in vowels_final:
    speaker,ideology,word,vowel,position = entry
    if (word in non_charged) or (word in charged):      
        c = word_speakers[word].get(speaker, 0)
        word_speakers[word][speaker] = c + 1

for word in word_speakers:
    word_count[word] = len(word_speakers[word])

In [18]:
word_count

{'ABOUT': 433,
 'ABSOLUTELY': 216,
 'AMENDMENT': 200,
 'AUTHORITY': 195,
 'BECAUSE': 456,
 'CALIFORNIA': 95,
 'CERTAINLY': 337,
 'CONGRESS': 282,
 'CONSTITUTION': 119,
 'CONSTITUTIONAL': 199,
 'DIFFERENT': 393,
 'ECONOMIC': 66,
 'EXACTLY': 277,
 'EXAMPLE': 303,
 'FEDERAL': 309,
 'GINSBURG': 258,
 'GOVERNMENT': 256,
 'HONOR': 364,
 'IMPORTANT': 314,
 'INTO': 383,
 'KENNEDY': 199,
 'LEGISLATIVE': 151,
 'LEGISLATURE': 85,
 'MR': 434,
 'OPPORTUNITY': 152,
 'OTHER': 435,
 'PARTICULAR': 301,
 'PEOPLE': 250,
 'POLICY': 176,
 'SCALIA': 269,
 'SITUATION': 294,
 'SPECIFICALLY': 235,
 'STATUTORY': 204,
 'UNDER': 436,
 'WHETHER': 421}

In [21]:
word_speakers['MR']

{'aaronmpanner': 2,
 'alanbmorrison': 2,
 'alanedwarduntereiner': 2,
 'alangura': 2,
 'alanmfreedman': 2,
 'alexanderfreichert': 2,
 'allisonmzieve': 2,
 'amandacleiter': 2,
 'amyhowe': 2,
 'amyzapp': 2,
 'andrewjpincus': 2,
 'andrewjrossman': 2,
 'andrewlfrey': 2,
 'anitaalvarez': 2,
 'annoconnell': 2,
 'anthonyayang': 2,
 'anthonyyang': 2,
 'arthurffergenson': 2,
 'arthurhbryant': 2,
 'barbarabmcdowell': 2,
 'barrybarnett': 2,
 'barryrostrager': 2,
 'beaubbrindley': 2,
 'berthdeixler': 2,
 'bethsbrinkmann': 2,
 'bradleysphillips': 2,
 'brianfbarov': 2,
 'brianplauten': 2,
 'brianshiffrin': 2,
 'brianwolfman': 2,
 'brucecsmith': 2,
 'brucejennisjr': 2,
 'brucejterris': 2,
 'caitlinjhalligan': 2,
 'candacecain': 2,
 'carolynakubitschek': 2,
 'cartergphillips': 2,
 'catherineestetson': 4,
 'catherinewsmith': 2,
 'cchristophertrower': 2,
 'chadareadler': 4,
 'charlesalanwright': 2,
 'charlesarothfeld': 2,
 'charlescondon': 2,
 'charlesjcooper': 2,
 'charlesmsevilla': 2,
 'charlesssims': 